# Import

In [117]:
import numpy as np
from random import choice, random, randint, sample
import copy

# Operations

In [118]:
OPERATIONS = [
    (np.add, 2, "({} + {})"),
    (np.subtract, 2, "({} - {})"),
    (np.divide, 2, "({} / {})"),
    (np.multiply, 2, "({} * {})"),
    (np.sin, 1, "sin({})"),
    (np.cos, 1, "cos({})"),
    (np.tan, 1, "tan({})"),
    (np.exp, 1, "exp({})"),
    (np.log, 1, "log({})"),
    (np.sqrt, 1, "sqrt({})"),
    (np.square, 1, "square({})"),
]

# Genothype definition

Let's consider the genotype as a list of elements with the following structure: [operator, first operand, secondo operand].
In this way we are creating a recursive function that computes a valid formula and represent it as a list (example: [ "+", ["sin", "x[0]" ], "x[1]"]).

In [119]:
def random_program(depth, input_dim):
    if depth == 0 or random() < 0.3:
        #Return a costant between 1 and 10 or a random variable (prob=0.3)
        if random() < 0.3:
            return str(randint(2, 10)) #costant
        else:
            return f"x[{randint(0, input_dim - 1)}]" #new leaf node

    op, arity, symbol = choice(OPERATIONS)
    children = [random_program(depth - 1, input_dim) for _ in range(arity)]
    return [symbol] + children  

## Transform the program into a human readable function

In [120]:
def program_to_string(program):
    if isinstance(program, str):  # leaf
        return program  
    elif isinstance(program, list):  
        try:
            _, _, symbol = next((op, arity, s) for op, arity, s in OPERATIONS if s == program[0])
        except StopIteration:
            raise ValueError(f"Not known operation: {program[0]}")
        
        children = [program_to_string(child) for child in program[1:]]
        
        return symbol.format(*children)


In [121]:
program_to_string(random_program(6, 3))

'tan(log(log(tan(sin((10 + x[2]))))))'

Now we need a function that given the genotype provide us with the output provided by the predicted function. This function must receive the input vector to perform his operation.

In [122]:
def evaluate_program(program, x):
    if isinstance(program, str):  # Leaf node
        #If it's a leaf, it could be a costant or a variable
        if program[0] == 'x':
            return x[int(program[2:-1])]
        else:
            return int(program)
    elif isinstance(program, list): 
        op = next(op for op, _, symbol in OPERATIONS if symbol == program[0])
        args = [evaluate_program(child, x) for child in program[1:]]
        try:
            return op(*args)
        except ZeroDivisionError:
            return np.inf

In [123]:
x = [2,3,4, 6]
program = ['({} / {})', 'x[1]', '5']
#evaluate_program returns y given the vector x and the program
evaluate_program(program, x)

np.float64(0.6)

As you may notice this function verify with the function __isinstance(element, type)__ if "element" is an instance of the "type", with the objective of understanding if it is a __leaf node__. If this is the case we simply extract the value out of it, otherwise we still need to invoke the function recursively.

## Fitness function

For now, simply consider the fitness function of a solution as it's mean square error compared to the expected results.

In [124]:
#Let's also introduce a fitness function that is the same used by professors
def fitness_evaluation_result(program, x, y):
    predictions = np.array([evaluate_program(program, x_row) for x_row in x.T])
    return float(100 * np.mean(np.square(predictions - y)))

def fitness_function(program, x, y):
    predictions = np.array([evaluate_program(program, x_row) for x_row in x.T])
    return np.mean((predictions - y) ** 2)

In [125]:
x = np.random.uniform(-1, 1, size=(2, 1000))  # 2 feature, 1000 esempi
y = x[0] + np.sin(x[1]) / 5
fitness_evaluation_result(program, x, y)

34.35118974410521

## Tweak function

There is a lot of __variability__ that has to be considered for the tweak function. We can now imagine to implement a recursive function that receives the program (which indicates the current function that we are using for the task), the number of dimensions for the input and the maximum depth allowed for the tweaked solution.

Recursively, if we end up into a leaf node, or if the current solution is still a list but with 0.3 probability, we simply generate a new sub-program.

Otherwise, we simply invoke the same function for a random index.

In [126]:
def mutate_program(program, input_dim, depth=3):
    if random() < 0.3 or not isinstance(program, list):  
        return random_program(depth, input_dim)
    idx = randint(1, len(program) - 1)
    program[idx] = mutate_program(program[idx], input_dim, depth - 1)
    return program

Other function (FIGP based)

In [127]:
def mutate_program(program, input_dim, max_depth=3):
    """Mutazione per rimpiazzare un sottoalbero casuale."""
    mutant = copy.deepcopy(program)
    
    def get_subtree_points(prog):
        return [i for i, node in enumerate(prog) if isinstance(node, list)]
    
    points = get_subtree_points(mutant)
    if not points:
        return random_program(max_depth, input_dim)  # Genera nuovo programma
    
    point = choice(points)
    new_subtree = random_program(randint(1, max_depth), input_dim)  # Nuovo sottoalbero
    mutant[point] = new_subtree
    
    return mutant


Other function (FIGP based)

In [128]:
#Try to swap one single operation with another one
def mutate(program, input_dim, max_depth=3):
    """Mutazione di un programma."""
    mutant = copy.deepcopy(program)

    def get_subtree_points(prog):
        return [i for i, node in enumerate(prog) if isinstance(node, list)]

    points = get_subtree_points(program)
    if not points:
        return random_program(max_depth, input_dim)  # Ritorna un nuovo programma se non ci sono punti

    point = choice(points)

    # Genera un nuovo sottoalbero casuale
    new_subtree = random_program(randint(1, max_depth), input_dim)
    mutant[point] = new_subtree

    return mutant

## Crossover

We can use a croossover function that receives only 2 parents and, if one of them is a leaf program simply return casually one of the 2 programs (avoiding to perform the operation for programs with no childrens). Otherwise, select random indexes for both the parents and combine the first part of the tree with the second part of the tree of the 2 parents, returning a new individual.

In [129]:
def crossover(parent1, parent2, max_depth=3):
    """Crossover tra due programmi."""
    child1, child2 = copy.deepcopy(parent1), copy.deepcopy(parent2)

    if len(parent1) < 2 or len(parent2) < 2:
        return parent1, parent2  # Evita crossover se i programmi sono troppo piccoli

    # Punti di crossover validi
    def get_subtree_points(prog):
        return [i for i, node in enumerate(prog) if isinstance(node, list)]

    points1 = get_subtree_points(parent1)
    points2 = get_subtree_points(parent2)

    if not points1 or not points2:
        return parent1, parent2  # Nessun punto valido per il crossover

    point1 = choice(points1)
    point2 = choice(points2)

    # Scambia i sottoalberi
    subtree1 = parent1[point1]
    subtree2 = parent2[point2]

    child1[point1], child2[point2] = subtree2, subtree1

    return child1, child2

New crossover (FIGP based):

In [130]:
def crossover(parent1, parent2):
    """Crossover a singolo punto tra due programmi."""
    child1, child2 = copy.deepcopy(parent1), copy.deepcopy(parent2)
    
    # Identifica punti di crossover validi
    def get_subtree_points(prog):
        return [i for i, node in enumerate(prog) if isinstance(node, list)]
    
    points1 = get_subtree_points(parent1)
    points2 = get_subtree_points(parent2)
    
    if not points1 or not points2:
        return parent1, parent2  # Nessun punto valido, restituisci genitori
    
    point1 = choice(points1)
    point2 = choice(points2)
    
    # Scambia sottoalberi selezionati
    subtree1 = parent1[point1]
    subtree2 = parent2[point2]
    child1[point1], child2[point2] = subtree2, subtree1
    
    return child1, child2


## Genetic algorithm

### Parameter selection

In [131]:
generations=100
population_size=100
offspring_size = 50

### Data selection

In [132]:
def true_f(x: np.ndarray) -> np.ndarray:
    return x[0] + np.tan(x[1])/5 

TEST_SIZE = 10_000
TRAIN_SIZE = 1000

x_validation = np.vstack(
    [
        np.random.random_sample(size=TEST_SIZE) * 2 * np.pi - np.pi,
        np.random.random_sample(size=TEST_SIZE) * 2 - 1,
    ]
)
y_validation = true_f(x_validation)
train_indexes = np.random.choice(TEST_SIZE, size=TRAIN_SIZE, replace=False)
x_train = x_validation[:, train_indexes]
y_train = y_validation[train_indexes]

x = x_train
y = y_train

Let's change the fitness function in a way that we penalize the more complex functions:

In [133]:
# Funzione di fitness avanzata ispirata a deep_based_FGP_NLS.py
def fitness_function(program, x, y):
    """
    Valuta la fitness di un programma.
    """
    try:
        # Valutazione del programma
        predictions = np.array([evaluate_program(program, x_row) for x_row in x.T])
        if np.any(np.isnan(predictions)) or np.any(np.isinf(predictions)):
            return np.inf  # Penalizza programmi invalidi

        # Calcolo dell'errore
        #error = np.mean((predictions - y) ** 2)  # Errore quadratico medio
        error = float(100 * np.mean(np.square(predictions - y)))
        

        # Aggiungi penalità basata sulla complessità del programma
        complexity_penalty = len(program) * 0.1
        fitness = error + complexity_penalty

    except Exception as e:
        # Penalizza programmi che generano errori
        print(f"Errore nella valutazione del programma: {e}")
        return np.inf

    return fitness

Let's call the previous fitness function "MSE":

In [134]:
def mse(program, x, y):
    predictions = np.array([evaluate_program(program, x_row) for x_row in x.T])
    return float(100 * np.mean(np.square(predictions - y)))

## Tournament selection for parents, tau set to 50

In [135]:
def tournament_selection(population,x,y, tau=5):
    tau = min(tau, len(population))
    
    # Seleziona tau individui casualmente dalla popolazione
    tournament_indices = np.random.choice(len(population), tau, replace=False)
    
    # Calcola i punteggi di fitness per gli individui selezionati nel torneo
    fitness_scores = [fitness_function(population[idx],x,y) for idx in tournament_indices]
    
    # Trova l'indice del miglior fitness nel torneo
    best_fitness_index = fitness_scores.index(min(fitness_scores))
    best_index = tournament_indices[best_fitness_index]
    
    # Ritorna il vincitore del torneo
    winner = population[best_index]
    
    return winner



## Simulated annealing

In [136]:
import math


def simulated_annealing(initial_program, x, y, max_iterations=500, initial_temperature=100, cooling_rate=0.95):
    
    # Programma corrente e relativa fitness
    current_program = initial_program
    current_fitness = fitness_function(current_program, x, y)
    
    # Memorizza il miglior programma trovato
    best_program = current_program
    best_fitness = current_fitness
    
    # Inizializza la temperatura
    temperature = initial_temperature
    
    for iteration in range(max_iterations):
        # Crea un programma candidato con una mutazione
        candidate_program = mutate_program(current_program, x.shape[0])
        candidate_fitness = fitness_function(candidate_program, x, y)
        
        # Calcola la variazione di fitness
        fitness_delta = candidate_fitness - current_fitness
        
        # Accetta il nuovo programma se migliora o con probabilità decrescente
        if fitness_delta < 0 or random() < math.exp(-fitness_delta / temperature):
            current_program = candidate_program
            current_fitness = candidate_fitness
            
            # Aggiorna il miglior programma trovato
            if current_fitness < best_fitness:
                best_program = current_program
                best_fitness = current_fitness
        
        # Riduci la temperatura
        temperature *= cooling_rate
        
        # Interrompi se la temperatura è troppo bassa
        if temperature < 1e-3:
            break
    
    return best_program

### Parameters

In [137]:
# Parametri del GP
generations = 200
population_size = 1000
p_crossover = 0.6
p_mutation = 0.4
tweak_probability = 0.2
max_depth = 3 #fixed or problem dependant?
elite_size = 100
offspring_size = population_size-elite_size # Numero di discendenti generati per generazione

Let's create a new tweak function that is able to tweak a program by adding an unary operation in leaf nodes (if it not a constant value).

In [138]:
def tweak_program_2(program):
    """
    Modifica un sottoalbero del programma aggiungendo un operatore unario
    su una foglia, con una certa probabilità.
    """
    # Trova tutte le foglie del programma
    def get_leaf_indices(node, path=()):
        if isinstance(node, str) or isinstance(node, (int, float)):
            # Nodo foglia (variabile o costante)
            return [path]
        elif isinstance(node, list) and len(node) > 1:
            # Nodo non foglia valido: esplora ricorsivamente i figli
            indices = []
            for i, child in enumerate(node[1:], start=1):
                indices.extend(get_leaf_indices(child, path + (i,)))
            return indices
        return []
    
    # Ottieni tutte le foglie del programma
    leaf_indices = get_leaf_indices(program)
    if not leaf_indices:
        return program  # Nessuna modifica possibile

    # Seleziona casualmente una foglia
    selected_leaf_path = choice(leaf_indices)
    
    # Verifica che il percorso selezionato sia valido
    if not selected_leaf_path:
        return program  # Nessuna modifica possibile

    # Accedi alla foglia selezionata
    node = program
    for idx in selected_leaf_path[:-1]:
        node = node[idx]

    # Verifica che il nodo sia valido prima di modificare
    if isinstance(node, list) and len(selected_leaf_path) > 0:
        leaf = node[selected_leaf_path[-1]]
        # Modifica la foglia con un operatore unario
        if isinstance(leaf, str) and leaf.startswith("x"):  # Se è una variabile
            unary_operator = choice(["sin({})", "cos({})", "tan({})", "log({})", "sqrt({})"])
            node[selected_leaf_path[-1]] = [unary_operator, leaf]
    
    return program


In [ ]:
# Inizializza popolazione
input_dim = x.shape[0]
# Ignora warning temporaneamente
np.seterr(all='ignore')
population = [random_program(max_depth, input_dim) for _ in range(population_size//2)]
while len(population)<population_size:
    #population.append(simulated_annealing(random_program(max_depth, input_dim), x, y))
    population.append(random_program(max_depth, input_dim))

# Loop principale per le generazioni
def run_genetic_algorithm():
    global population

    for gen in range(generations):
        # Calcola la fitness per ogni individuo
        fitness = np.array([fitness_function(prog, x, y) for prog in population])
        mse_value = np.array([mse(prog, x, y) for prog in population])

        # Mantieni l'elite (migliori individui)
        sorted_indices = fitness.argsort()
        elite = [population[i] for i in sorted_indices[:100]]  # Manteniamo 2 migliori
        best_fitness = fitness[sorted_indices[0]]
        mse_to_print = mse_value[sorted_indices[0]]
        #break the cycle if you found the best solution you're able to find with training data
        if(mse_to_print==0.0000): #use mse not to consider complexity penalty
            # Riporta i warning al comportamento normale
            np.seterr(all='warn')
            print('Best program found with mse=0')
            return population[sorted_indices[0]]
            # Ignora warning temporaneamente
            np.seterr(all='ignore')
        # Riporta i warning al comportamento normale
        np.seterr(all='warn')
        print(f"Generazione {gen + 1}, miglior fitness: {mse_to_print:.6f}")
        #best formula found
        print(f"Best formula: {program_to_string(population[sorted_indices[0]])}")
        # Ignora warning temporaneamente
        np.seterr(all='ignore')
        # Crea la nuova generazione
        next_population = []

        while len(next_population) < offspring_size:
            if random() < p_crossover and len(population) > 200:
                # Crossover
                #With random choice is much faster than tournament selection
                #Choose the best and the second best parent
                parent1, parent2 = tournament_selection(population,x,y), tournament_selection(population,x,y)
                child1, child2 = crossover(parent1, parent2)

                if random() < p_mutation:
                    child1 = mutate(child1, input_dim)
                if random() < p_mutation:
                    child2 = mutate(child2, input_dim)

                if random() < tweak_probability and len(next_population) < offspring_size:
                    new_ind = tweak_program_2(child1)
                    next_population.append(new_ind)
                if random() < tweak_probability and len(next_population) < offspring_size:
                    new_ind = tweak_program_2(child2)
                    next_population.append(new_ind)
                if len(next_population) < offspring_size:
                    next_population.append(child1)
                if len(next_population) < offspring_size:
                    next_population.append(child2)
            else:
                # Mutazione diretta di un genitore
                #Use tournament selection also here, but only if there's enough material, otherwise random
                if(len(population)>500):
                    parent = tournament_selection(population,x,y)
                else:
                    parent = choice(population)
                mutant = mutate(parent, input_dim)
                if len(next_population) < offspring_size:
                    next_population.append(mutant)

        # Combina elite e nuova generazione
        sorted_indices = fitness.argsort()
        elite = [population[i] for i in sorted_indices[:100]]  # Manteniamo 2 migliori
        next_population.extend(elite)

        # Rimuovi duplicati
        unique_population = {}
        for prog in next_population:
            serialized = str(prog)
            if serialized not in unique_population:
                unique_population[serialized] = prog

        # Aggiorna la popolazione con individui unici
        population = list(unique_population.values())[:population_size]

    # Identifica il miglior programma
    fitness = np.array([fitness_function(prog, x, y) for prog in population])
    mse_value = np.array([mse(prog, x, y) for prog in population])
    sorted_indices = fitness.argsort()
    best_program = population[sorted_indices[0]]
    best_fitness = fitness[sorted_indices[0]]
    best_mse = mse_value[sorted_indices[0]]

    print("Miglior programma:", best_program, "; Fitness:", best_mse)
    return best_program

best_program = run_genetic_algorithm()


Generazione 1, miglior fitness: 2.353256
Best formula: (x[0] + ((x[1] / x[0]) / exp(10)))
Generazione 2, miglior fitness: 2.353256
Best formula: (x[0] + ((x[1] / x[0]) / exp(10)))
Generazione 3, miglior fitness: 2.353190
Best formula: (x[0] / (x[0] / x[0]))
Generazione 4, miglior fitness: 0.653501
Best formula: (x[0] + (x[1] / (2 * 4)))
Generazione 5, miglior fitness: 0.327271
Best formula: (x[0] + (x[1] / 6))
Generazione 6, miglior fitness: 0.152806
Best formula: (x[0] + (x[1] / 5))
Generazione 7, miglior fitness: 0.152806
Best formula: (x[0] + (x[1] / 5))
Generazione 8, miglior fitness: 0.152806
Best formula: (x[0] + (x[1] / 5))
Generazione 9, miglior fitness: 0.152806
Best formula: ((x[1] / 5) + x[0])
Generazione 10, miglior fitness: 0.152806
Best formula: ((x[1] / 5) + x[0])
Generazione 11, miglior fitness: 0.152806
Best formula: (x[0] + (x[1] / 5))
Generazione 12, miglior fitness: 0.152806
Best formula: ((x[1] / 5) + x[0])


In [ ]:
program_to_string(best_program)

'((x[1] / 5) + x[0])'

In [ ]:
for program in population:
    print(program_to_string(program))

(x[0] + square(x[1]))
(x[0] + (exp(3) - exp(x[0])))
(x[0] - (x[0] - (x[0] + x[1])))
(x[0] + (x[0] - x[1]))
(x[0] + cos(square(2)))
(sin(9) + x[0])
(x[0] + sqrt(x[0]))
(x[0] + square(cos(x[1])))
((sin(x[0]) - cos(9)) + x[0])
((square(x[1]) * exp(x[1])) + x[0])
((2 * x[1]) + x[0])
(x[0] - 4)
(x[0] - x[1])
(x[0] - ((x[1] / 8) * x[1]))
(sqrt(9) + x[0])
(x[0] + sin(3))
(x[0] + x[0])
(x[0] - tan((x[0] / 7)))
(sin((sqrt(9) + (x[1] - x[0]))) - exp(x[0]))
(x[0] + tan(sin(3)))
(square(log(5)) + x[0])
(x[0] - (square(x[1]) - x[1]))
((log(x[1]) * log(x[1])) + x[0])
((sin(2) + (x[0] - x[0])) + x[0])
(x[0] / tan((6 * sin(x[1]))))
(x[0] + sin(6))
tan((sqrt(x[0]) + x[0]))
(x[0] - log((x[1] * x[0])))
(7 + x[0])
(x[0] - square(sin(6)))
(10 + x[0])
(x[0] + sqrt(5))
(x[0] / cos((sin(x[1]) * exp(x[0]))))
(sin(sin(x[0])) + x[1])
(x[0] + ((2 - 7) / 3))
(x[0] + (7 * x[0]))
(x[0] + (x[1] / 9))
((x[1] * x[0]) / 10)
(x[0] - ((3 + 3) + cos(x[0])))
((x[1] + x[0]) + sqrt((x[1] - x[1])))
(sin(3) + x[0])
(x[0] - (x[1